In [2]:
import numpy as np
import torch 
import sys
sys.path.append("../")

import tqdm
import yaml
torch.random.manual_seed(300)
import matplotlib.pyplot as plt

import utils_BayTIDE as utils

from model_BayTIDE import BayTIDE

from model_LDS import LDS_GP_streaming

config_path = "./config_uber_sensitive.yaml"
with open(config_path, "r") as f:
    config = yaml.safe_load(f)

data_file = config["data_path"] # T=315 - rmse = 0.27 - Mqar23

hyper_dict = utils.make_hyper_dict(config)

data_dict = utils.make_data_dict(hyper_dict,data_file,fold=1)

model = BayTIDE(hyper_dict,data_dict)

model.reset()

INNER_ITER = hyper_dict["INNER_ITER"]
EVALU_T = hyper_dict["EVALU_T"]
for epoch in range(1):
    model.reset()
    for T_id in tqdm.tqdm(range(model.T)):
        model.filter_predict(T_id)
        model.msg_llk_init()

        for inner_it in range(INNER_ITER):

            flag = (inner_it == (INNER_ITER - 1))

            model.msg_approx_W(T_id)
            model.post_update_W(T_id)

            model.msg_approx_U(T_id)
            model.filter_update(T_id,flag)

        model.msg_approx_tau(T_id)
        model.post_update_tau(T_id)

        if T_id % EVALU_T == 0 or T_id == model.T - 1:
            
            # _, loss_dict = model.model_test(T_id)
            # print("T_id = {}, train_rmse = {:.2f}, test_rmse= {:.2f}".format(T_id, loss_dict["train_RMSE"], loss_dict["test_RMSE"]))
            pass

    model.smooth()
    model.post_update_U_after_smooth(0)
    _, loss_dict = model.model_test(T=T_id, prob=True)
    fold_id=0
    print("fold = {}, after smooth: \n test_rmse= {:.3f}, \n test_MAE= {:.3f}, \n CRPS= {:.3f},\n neg-llk= {:.3f}".format(fold_id,loss_dict["test_RMSE"], loss_dict["test_MAE"],loss_dict["CRPS"], loss_dict["neg-llk"]))



use device: cuda


100%|██████████| 744/744 [00:07<00:00, 94.98it/s]


tensor(0.0739, dtype=torch.float64)
fold = 0, after smooth: 
 test_rmse= 2.279, 
 test_MAE= 1.626, 
 CRPS= 0.074,
 neg-llk= 2.260
